In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report


import pandas as pd
from sklearn.model_selection import train_test_split

# Carregar os dados
dados = pd.read_parquet("TMDB_movie_dataset_v11.parquet")

# Selecionar colunas relevantes
dados = dados[["release_date", "revenue", "budget", "runtime", "genres", 
              "original_language", "production_countries", "spoken_languages", "adult"]]

# Filtrar dados (adult == False)
dados = dados[dados["adult"] == False].copy()

# Criar coluna 'disponibilidade_lucro'
dados["disponibilidade_lucro"] = (
    dados["revenue"].apply(lambda x: isinstance(x, (int, float)) and x > 0) & 
    dados["budget"].apply(lambda x: isinstance(x, (int, float)) and x > 0)
).astype(int)

# Criar conjunto de teste e treino (mantendo proporção original para o treino)
dados_teste, dados_treino = train_test_split(
    dados, test_size=0.8, stratify=dados["disponibilidade_lucro"], random_state=42
)

# ----- AJUSTE DO CONJUNTO DE TESTE (80/20) -----
classe_1_teste = dados_teste[dados_teste["disponibilidade_lucro"] == 1]
classe_0_teste = dados_teste[dados_teste["disponibilidade_lucro"] == 0]

# Selecionar 70% dos registros da classe 1 no conjunto de teste original
qtd_classe_1_novo_teste = int(len(classe_1_teste) * 0.7)
classe_1_novo_teste = classe_1_teste.sample(n=qtd_classe_1_novo_teste, random_state=42, replace=False)

# Definir o total necessário no novo conjunto de teste (80/20)
total_novo_teste = qtd_classe_1_novo_teste * 5  

# Definir a quantidade de registros 0 necessária
qtd_classe_0_novo_teste = min(total_novo_teste - qtd_classe_1_novo_teste, len(classe_0_teste))

# Amostrar os registros da classe 0
classe_0_teste_balanceada = classe_0_teste.sample(n=qtd_classe_0_novo_teste, random_state=42, replace=False)

# Criar o novo conjunto de teste balanceado
dados_teste_balanceado = pd.concat([classe_1_novo_teste, classe_0_teste_balanceada]).sample(frac=1, random_state=42).reset_index(drop=True)

# ----- AJUSTE DO CONJUNTO DE TREINO (50/50) -----
classe_1_treino = dados_treino[dados_treino["disponibilidade_lucro"] == 1]
classe_0_treino = dados_treino[dados_treino["disponibilidade_lucro"] == 0]

# Selecionar 70% dos registros da classe 1 no conjunto de treino original
qtd_classe_1_novo_treino = int(len(classe_1_treino) * 0.7)
classe_1_novo_treino = classe_1_treino.sample(n=qtd_classe_1_novo_treino, random_state=42, replace=False)

# Para balanceamento 50/50, a classe 0 precisa ter a mesma quantidade da classe 1
qtd_classe_0_novo_treino = qtd_classe_1_novo_treino

# Amostrar os registros da classe 0
classe_0_treino_balanceada = classe_0_treino.sample(n=qtd_classe_0_novo_treino, random_state=42, replace=False)

# Criar o novo conjunto de treino balanceado
dados_treino_balanceado = pd.concat([classe_1_novo_treino, classe_0_treino_balanceada]).sample(frac=1, random_state=42).reset_index(drop=True)

# ---- PRINTS ADICIONAIS ----
print("\n---- Conjunto de Teste Balanceado (80/20) ----")
print("Número total de registros:", len(dados_teste_balanceado))
print("Número de registros da classe 1:", len(classe_1_novo_teste))
print("Número de registros da classe 0:", len(classe_0_teste_balanceada))
print("Proporção de classes no conjunto de teste:")
print(dados_teste_balanceado["disponibilidade_lucro"].value_counts(normalize=True))

print("\n---- Conjunto de Treino Balanceado (50/50) ----")
print("Número total de registros:", len(dados_treino_balanceado))
print("Número de registros da classe 1:", len(classe_1_novo_treino))
print("Número de registros da classe 0:", len(classe_0_treino_balanceada))
print("Proporção de classes no conjunto de treino:")
print(dados_treino_balanceado["disponibilidade_lucro"].value_counts(normalize=True))



---- Conjunto de Teste Balanceado (80/20) ----
Número total de registros: 9810
Número de registros da classe 1: 1962
Número de registros da classe 0: 7848
Proporção de classes no conjunto de teste:
disponibilidade_lucro
0    0.8
1    0.2
Name: proportion, dtype: float64

---- Conjunto de Treino Balanceado (50/50) ----
Número total de registros: 15692
Número de registros da classe 1: 7846
Número de registros da classe 0: 7846
Proporção de classes no conjunto de treino:
disponibilidade_lucro
1    0.5
0    0.5
Name: proportion, dtype: float64


In [16]:
dados_teste = dados_teste_balanceado.copy()

dados_treino_filtrado = dados_treino_balanceado.copy()

for df in [dados_treino_filtrado, dados_teste]:
    df['lucro'] = df.apply(lambda row: row['revenue'] - row['budget']
                           if isinstance(row['budget'], (int, float)) and isinstance(row['revenue'], (int, float))
                           else None, axis=1)

    df['classificacao'] = df.apply(lambda row: 1 if row['lucro'] is not None and row['lucro'] > 0 and row['disponibilidade_lucro'] == 1 else 0, axis=1)

In [17]:

# Criar novas colunas (número de idiomas e idade)
for df in [dados_treino_filtrado, dados_teste]:
    df["num_languages"] = df["spoken_languages"].apply(lambda x: len(x.split(",")) if isinstance(x, str) else 0)
    df["idade"] = df["release_date"].apply(lambda x: 2025 - int(x.split("-")[0]) if isinstance(x, str) else 0)

# Lista de gêneros
generos = ['Action', 'Science Fiction', 'Adventure', 'Drama', 'Crime',
           'Thriller', 'Fantasy', 'Comedy', 'Romance', 'Western', 'Mystery', 'War',
           'Animation', 'Family', 'Horror', 'Music']

def aplicar_one_hot_encoding_generos(df, generos):
    for genero in generos:
        df[genero] = df['genres'].apply(lambda x: 1 if isinstance(x, str) and genero in x.split(', ') else 0)
    return df

# Aplicar one-hot encoding de gêneros
dados_treino_filtrado = aplicar_one_hot_encoding_generos(dados_treino_filtrado, generos)
dados_teste = aplicar_one_hot_encoding_generos(dados_teste, generos)

def aplicar_one_hot_encoding_limitado(df, idiomas_permitidos, selected_countries):
    df['original_language_encoded'] = df['original_language']
    df['production_countries_encoded'] = df['production_countries']
    df['original_language_encoded'] = df['original_language_encoded'].apply(
        lambda x: x if x in idiomas_permitidos else 'other_language'
    )
    df['production_countries_encoded'] = df['production_countries_encoded'].apply(
        lambda x: x if x in selected_countries else 'other_country'
    )
    df = pd.get_dummies(df, columns=['original_language_encoded', 'production_countries_encoded'], prefix=['lang', 'country'])
    return df

# Listas de idiomas e países permitidos
idiomas_permitidos = ['en', 'fr', 'es', 'de', 'ja', 'zh', "pt", 'it']
selected_countries = ['United States', 'France', 'United Kingdom', 'Germany', 
                      'Canada', 'Japan', 'China', 'India', 'Italy', 'Spain']

# Aplicar one-hot encoding limitado
dados_treino_encoded = aplicar_one_hot_encoding_limitado(dados_treino_filtrado, idiomas_permitidos, selected_countries)
dados_teste_encoded = aplicar_one_hot_encoding_limitado(dados_teste, idiomas_permitidos, selected_countries)

# Garantir que a coluna 'genres' seja preservada
dados_treino_encoded['genres'] = dados_treino_filtrado['genres']
dados_teste_encoded['genres'] = dados_teste['genres']

# Alinhar colunas
dados_treino_encoded, dados_teste_encoded = dados_treino_encoded.align(dados_teste_encoded, join='outer', axis=1, fill_value=0)

dados_treino_encoded.to_parquet("dados_treino.parquet", index=False)
dados_teste_encoded.to_parquet("dados_teste.parquet", index=False)


In [18]:
dados_teste_encoded.columns

Index(['release_date', 'revenue', 'budget', 'runtime', 'genres',
       'original_language', 'production_countries', 'spoken_languages',
       'adult', 'disponibilidade_lucro', 'lucro', 'classificacao',
       'num_languages', 'idade', 'Action', 'Science Fiction', 'Adventure',
       'Drama', 'Crime', 'Thriller', 'Fantasy', 'Comedy', 'Romance', 'Western',
       'Mystery', 'War', 'Animation', 'Family', 'Horror', 'Music', 'lang_de',
       'lang_en', 'lang_es', 'lang_fr', 'lang_it', 'lang_ja',
       'lang_other_language', 'lang_pt', 'lang_zh', 'country_Canada',
       'country_China', 'country_France', 'country_Germany', 'country_India',
       'country_Italy', 'country_Japan', 'country_Spain',
       'country_United Kingdom', 'country_other_country'],
      dtype='object')

In [19]:
dados_treino_encoded.columns

Index(['release_date', 'revenue', 'budget', 'runtime', 'genres',
       'original_language', 'production_countries', 'spoken_languages',
       'adult', 'disponibilidade_lucro', 'lucro', 'classificacao',
       'num_languages', 'idade', 'Action', 'Science Fiction', 'Adventure',
       'Drama', 'Crime', 'Thriller', 'Fantasy', 'Comedy', 'Romance', 'Western',
       'Mystery', 'War', 'Animation', 'Family', 'Horror', 'Music', 'lang_de',
       'lang_en', 'lang_es', 'lang_fr', 'lang_it', 'lang_ja',
       'lang_other_language', 'lang_pt', 'lang_zh', 'country_Canada',
       'country_China', 'country_France', 'country_Germany', 'country_India',
       'country_Italy', 'country_Japan', 'country_Spain',
       'country_United Kingdom', 'country_other_country'],
      dtype='object')